In [2]:
# prediction interval for mlps on the housing regression dataset
from numpy import asarray
from pandas import read_csv
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam


In [3]:
# load and prepare the dataset
def load_dataset(url):
	dataframe = read_csv(url, header=None)
	values = dataframe.values
	# split into input and output values
	X, y = values[:, :-1], values[:,-1]
	# split into train and test sets
	X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.67, random_state=1)
	# scale input data
	scaler = MinMaxScaler()
	scaler.fit(X_train)
	X_train = scaler.transform(X_train)
	X_test = scaler.transform(X_test)
	return X_train, X_test, y_train, y_test


In [4]:
# define and fit the model
def fit_model(X_train, y_train):
	# define neural network model
	features = X_train.shape[1]
	model = Sequential()
	model.add(Dense(20, kernel_initializer='he_normal', activation='relu', input_dim=features))
	model.add(Dense(5, kernel_initializer='he_normal', activation='relu'))
	model.add(Dense(1))
	# compile the model and specify loss and optimizer
	opt = Adam(learning_rate=0.01, beta_1=0.85, beta_2=0.999)
	model.compile(optimizer=opt, loss='mse')
	# fit the model on the training dataset
	model.fit(X_train, y_train, verbose=0, epochs=300, batch_size=16)
	return model



In [5]:
# fit an ensemble of models
def fit_ensemble(n_members, X_train, X_test, y_train, y_test):
    ensemble = list()
    for i in range(n_members):
        model = fit_model(X_train, y_train)
        yhat = model.predict(X_test, verbose=0)
        mae = mean_absolute_error(y_test, yhat)
        ensemble.append(model)
        return ensemble

In [19]:
# make predictions with the ensemble and calculate a prediction interval
def predict_with_pi(ensemble, X):
    yhat = [model.predict(X, verbose=0) for model in ensemble]
    yhat = asarray(yhat)
    print(yhat)
    
    # calculate 95% prediction interval
    interval = 1.96 * yhat.std()
    lower, upper = yhat.mean() - interval, yhat.mean() + interval
    return lower, yhat.mean(), upper


In [20]:
# load dataset
url = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/housing.csv'
X_train, X_test, y_train, y_test = load_dataset(url)

# fit ensemble
n_members = 30
ensemble = fit_ensemble(n_members, X_train, X_test, y_train, y_test)

# make predictions with prediction interval
newX = asarray([X_test[0, :]])
lower, mean, upper = predict_with_pi(ensemble, newX)

print('Point prediction: %.3f' % mean)
print('95%% prediction interval: [%.3f, %.3f]' % (lower, upper))
print('True value: %.3f' % y_test[0])

>1, MAE: 2.641
>2, MAE: 2.443
>3, MAE: 2.496
>4, MAE: 2.198
>5, MAE: 2.370
>6, MAE: 2.570
>7, MAE: 2.194
>8, MAE: 2.319
>9, MAE: 2.488
>10, MAE: 2.256
>11, MAE: 2.294
>12, MAE: 2.648
>13, MAE: 2.268
>14, MAE: 2.556
>15, MAE: 2.730
>16, MAE: 2.413
>17, MAE: 6.622
>18, MAE: 2.430
>19, MAE: 2.360
>20, MAE: 2.441
>21, MAE: 2.552
>22, MAE: 2.755
>23, MAE: 2.314
>24, MAE: 2.446
>25, MAE: 2.331
>26, MAE: 2.273
>27, MAE: 2.405
>28, MAE: 2.561
>29, MAE: 2.532
>30, MAE: 2.391
[[[31.576057]]

 [[32.72825 ]]

 [[29.75198 ]]

 [[30.956024]]

 [[30.135996]]

 [[30.844135]]

 [[31.268238]]

 [[29.670761]]

 [[32.48204 ]]

 [[30.12108 ]]

 [[30.158148]]

 [[33.490852]]

 [[28.442892]]

 [[31.485723]]

 [[31.394283]]

 [[29.786516]]

 [[22.26171 ]]

 [[32.293217]]

 [[28.65356 ]]

 [[31.100992]]

 [[30.947462]]

 [[31.238157]]

 [[28.684875]]

 [[31.570925]]

 [[31.400743]]

 [[31.569508]]

 [[34.306564]]

 [[29.810005]]

 [[30.066046]]

 [[30.164003]]]
Point prediction: 30.612
95% prediction interval:

In [21]:
mean

30.612026